## Problem Statement - Identification of factors for varying car price and help Chinese company to enter US market.

## Steps to follow
- Data Loading and preliminary understanding the data
- Data Correction
- Understand the data and perform EDA on the data
- Split the data into train and test set for Linear regression
- Build Model and Perform train set validation
- Use different combination of Independent variable to perform validation of train results
    - Correlation analysis
    - Check P-values for different combination
    - Evaluation of R Square, Adjusted R Square and VIF 
- Indentification of Key Factors based on R2, Adjusted R2 and VIF
- Predict on Test set
- Compare Train and Test R2-Square value

In [ ]:
# ALL IMPORTS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

### Utility Functions

In [ ]:
'''
For Displaying Value on top of the bar plots
'''
def val_on_bar(axs,frmt=' '):
    def _single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height()
            value = '{:.2f}'.format(p.get_height()) + frmt
            ax.text(_x, _y, value , ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single_plot(ax)
    else:
        _single_plot(axs)

'''
Function For Lable Encoding Not used
'''
from sklearn.preprocessing import LabelEncoder

def performEncoding(encodingType,df):
    
    if encodingType=='LE':
        le = LabelEncoder() 
        tmp = df
        #Use Label Encoder for These columns as having many categorical variables and having huge class imbalance
        # We can scale them using min max scalar to have the values between 0 and 1
        # Later by using RFE we can remove the features which are not havint statistical significance
        #'enginelocation' - rop from analysis
        le_cols = []
        for col in le_cols:
            le.fit(df[col].drop_duplicates()) 
            df[col] = le.transform(tmp[col])
    
    return df

# Step 1 : Loading and Understanding of the data
- Load the data - Car data and data dictionary for car price
- Correct the data corresponding to data dictionary xls - Lot many null columns, unwanted columns as unnamed etc
- Create a dataframe containing the column Name, data type and column desription mapped to a single record od car data
- Derived Matrix for Car Company Name
    - Correct Car Company Name
- Explore the car data based on various Plots
- Check the data using Pairplot, Heatmap

In [ ]:
#Load Car Price Data

car = pd.read_csv("CarPrice_Assignment.csv")

# Load the data dictionary

data_dict = pd.read_excel("Data Dictionary - carprices.xlsx")

#Correction of Data Dictionary Dataframe

data_dict.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5',
                'Unnamed: 6','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 12','Unnamed: 13'],axis=1,inplace=True)
data_dict.dropna(axis=0,inplace=True)
data_dict[['Col_Name','Decription']] = data_dict
data_dict.drop(['Unnamed: 7','Unnamed: 11'],axis=1,inplace=True)
data_dict.reset_index(inplace=True)
data_dict.drop('index',axis=1,inplace=True)

'''
Create the Dataframe for dtypes
'''
d_typ = pd.DataFrame(car.dtypes,columns=['dtypes'])
#Reset the Index
d_typ = d_typ.reset_index()
#Rename the Index column as Column_name
d_typ['column_name'] = d_typ['index']
d_typ = d_typ[['column_name','dtypes']]

'''
Create A New Column with First Value/Any one value from the Main Dataframe Lending_loan
'''
d_typ['Column value'] = car.loc[0].values

'''
Concat With Data_Dictionary to have a view of the data mapped with Column Name and Description
'''

data_view = pd.concat([d_typ,data_dict],axis=1)
data_view

## Step 2: Data Correction and Preparation
### A-  Check For Null and NA values in the Dataset

In [ ]:
nadata = False
# Check for Null Data in the Car data
if len(car.columns[car.isna().any()]) ==0:
    print('There is no null data in the Car dataset')
else:
    print('Deal with Null Data')
    nadata =True
    
if len(car.columns[car.isnull().any()])== 0:
    print('There is no NA data in the Car dataset')
else:
    print('Deal with NA Data')
    
if nadata:
    missing_data = car.isnull().sum()
    missing_data.sort_values(inplace=True, ascending=False)
    print(missing_data)

### B-  Data Preparation

There is a variable named CarName which is comprised of two parts - the first word is the name of 'car company' and the second is the 'car model'. For example, chevrolet impala has 'chevrolet' as the car company name and 'impala' as the car model name. 


As per problem statement, We need to consider only company name as the independent variable for model building.

In the dataset, for few of the records the car company name is not correct or short code is used. Correct the car company name to have it nice categorised.

In [ ]:
'''
1 - Get Car Company Name
'''

car['companyName']= car.CarName.apply(lambda x: x.split()[0])
car['companyName'].unique()

'''
2 - Correct Company Name
'''
#Correct Incorrect Car Compnay Names 
    # - toyouta : toyota, vokswagen: volkswagen, vw: volkswagen, maxda : mazda, nissan: Nissan, porcshce: porsche

company_mapping = {"companyName":{ "maxda": 'mazda','porcshce':'porsche', "toyouta": 'toyota',
                                  "vokswagen": 'volkswagen',"vw": 'volkswagen','Nissan':'nissan'}}
car = car.replace(company_mapping)

'''
3 - Drop Car Name Column and Car_id
'''
car.drop(['CarName','car_ID'],axis=1,inplace=True)


#'''
#4 - Convert Symboling to object type datatype
#'''
#Its assigned insurance risk rating, A value of +3 indicates that the auto is risky, 
# -3 that it is probably pretty safe.(Categorical)

# car['symboling'] = car['symboling'].astype(object, copy=True)

### There are duplicate row records with different price

In [ ]:
except_price = list(car.columns)
except_price.remove('price')
car_X = car[except_price]
car = car[~car_X.duplicated()] ## Duplicate Features but different amount
#Display the dataframe duplicate records 
car_X[car_X.duplicated()]

Finding :
    - Excluding the Car Model Name and creating Company name column, if we exclude the dependent column 'Price' then there have few duplicate records(identical columns). So remove the duplicate records.

### Outlier Detection based on 3-Sigma Rule
- When checked the dataset, identifed outliners with respect to Horsepower, peakrpm and price by applying 3-sigma rule.
- As we are having very minimum data and these may be actual data, with some exceptional Hosepower and Peakrmp configuration or Price may be high due to some other properties, we can not exclude them from analysis.

In [ ]:
#Checking for outlier based on horsepower
car[car['horsepower'] > 1000].count()

#Checking for outlier data based on horsepower
car[np.abs(car.horsepower-car.horsepower.mean())>=(3*car.horsepower.std())]

In [ ]:
#Checking for outlier data based on horsepower
car[np.abs(car.peakrpm-car.peakrpm.mean())>=(3*car.peakrpm.std())]

In [ ]:
#Checking for outlier data based on price
car[np.abs(car.price-car.price.mean())>=(3*car.price.std())]

Finding:
    - As we will be using MIN_MAX_SCALER, we don't have to treat the Outliers. After min max scaler all the values will be within the range 0 and 1. Hence DO NOT DROP THEM.

## Step 3 - Exploratory Data Analysis

### A - Check the Categorical data - Count of each category and their percentages

Huge class imbalance between categories held within a single features such as: 
    - symboling(-2) has only 1.4% of data
    - carbody - hardtop  - 3.9%,convertible 2.9%
    - drivewheel- 4wd    - 4.3%
    - rear enginelocation - 1.4%
    - engine type rotor - 1.9%, dohcv- .4%
    - cylindernumber - eight, two, twelve, three are having less than 5% of data in each category
    - fuelsystem - spdi, 4bbl, mfi, spfi are having less than 5% of data in each category
    - companyName  - dodge, bmw, buick, plymouth, audi, saab, porsche, isuzu, jaguar, alfa-romero, chevrolet, renault, mercury are having less than 5% of data in each category

- So, let's first perform EDA on the data then later we will encode the categorical features for Model Building.
- We will deal with all the categorical features differently depending on there distributions

In [ ]:
'''
Check the Categorical data - Count of each category and their percentages
'''
cat_car = car.select_dtypes(include=['object'])
for col in cat_car.columns:
    print(col,':')
    print(pd.concat([cat_car[col].value_counts(),cat_car[col].value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))
    print('----------------------------------------------\n')
    

### Univariate Analysis
- Number of vehicles by company name
- Car Insurance Risk Factor - symboling
- Aspiration and Fueltype - pie chart
- No of doors, Drive wheel and carbody type bar plots 

In [ ]:
car['companyName'].value_counts().nlargest(car['companyName'].count()).plot(kind='bar', figsize=(15,5))
plt.title("Number of vehicles by company name")
plt.ylabel('Number of vehicles')
plt.xlabel('Company Name');

Findings: 
    - The number of cars under each car company is not sampled uniformly. There is huge class imbalance.

In [ ]:
plt.subplot(1,3,1)
car.symboling.value_counts().plot(kind='bar',figsize=(12, 4))
plt.title("Symboling Bar Plot")
plt.ylabel('Number of vehicles')
plt.xlabel('Symboling type');

plt.subplot(1,3,2)
car['aspiration'].value_counts().plot.pie(figsize=(12, 4), autopct='%.2f')
plt.title("Aspiration pie diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Aspiration type');

plt.subplot(1,3,3)
car['fueltype'].value_counts().plot.pie(figsize=(12, 4), autopct='%.2f')
plt.title("Fuel type pie diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Fuel type');

Findings: 
    - The number of cars under each risk factor is not sampled uniformly. There is huge class imbalance.
    - Aspiration type 'std' is having 81.73% of the data where as 'turbo' is 18.27
    - Fuel Type gas- 90.36 where as diesel-9.64
    - We will deal with the categorical variable after performing the analysis on theirdistribution.

In [ ]:
plt.figure(figsize=(24,6))
plt.subplot(1,4,1)
car['doornumber'].value_counts().plot(kind='bar')
plt.title("Car doors frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Number of doors');

plt.subplot(1,4,2)
car['drivewheel'].value_counts().plot(kind='bar')
plt.title("Drive wheel frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Drivewheel Type');

plt.subplot(1,4,3)
car['carbody'].value_counts().plot(kind='bar')
plt.title("Carbody type frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Carbody type');

plt.subplot(1,4,4)
car['cylindernumber'].value_counts().plot(kind='bar')
plt.title("Cylinder Number type frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Cylinder Number');

Finding:
    - Drivewheel, carbody and cylinder do not have even distribution. We need to perform appropriate encoding to use them in model building.
    - Fore Wheeldrive and Rear Wheel Drive are more than fur wheel drive
    - Sedan and Hatchback are more than other categories
    - Four cylindercars are more than other categories

In [ ]:
plt.figure(figsize=(24,6))
plt.subplot(1,2,1)
car['enginetype'].value_counts().plot(kind='bar')
plt.title("enginetype frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Engine type');

plt.subplot(1,2,2)
car['fuelsystem'].value_counts().plot(kind='bar')
plt.title("fuelsystem frequency diagram")
plt.ylabel('Number of vehicles')
plt.xlabel('Fuel System Type');

Findings:
    - Car doors can be converted to numerical 4 for 'four' and 2 for 'two'
    - Drivewheel, carbody type, engine type and fuel system type are having quite less nos of data for few categories. This is also an example case of undersampling.
    - cylinder number can be converted to numerical values i.e four -4, six -6 etc.

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
car['curbweight'].hist(bins=5,color='green');
plt.title("Curb weight histogram")
plt.ylabel('Number of vehicles')
plt.xlabel('Curb weight');

plt.subplot(1,3,2)
car['horsepower'].hist(bins=5,color='blue');
plt.title("Horsepower histogram")
plt.ylabel('Number of vehicles')
plt.xlabel('Horsepower');

plt.subplot(1,3,3)
car['enginesize'].hist(bins=5,color='blue');
plt.title("Enginesize histogram")
plt.ylabel('Number of vehicles')
plt.xlabel('Enginesize');

Findings:
    - There are quite less number of cars in higher curbweight, horsepower and enginesize

### Bivariate and Correlation Analysis
- Box Plots - Car distribution based on various categorical features
- Grouping of features and checking the mean price

##### Price distribution based on Categorical Features - using box plots
 - Analysis Summary : 
     - Fuel Type: Diesel cars are on an average higher price than Gas Cars - But High Price Car are of type Gas. Class imbalance problem.
     - Turbo aspiration cars are on an average higher price than std Cars - But most of the high price car are of type std. Class imbalance problem. We need more data to know the actual affect.
     - Carbody - hardtop cars are on an average higher price than others - But Less Cars of type Hardtop - Class imbalance problem. We need more data to know the actual affect.
     - Drivewheel of type 'rwd' affects the increase in Price type -  it seems rear-wheel drive vehicles are, on an average, the most expensive, while 4-wheel and front-wheel are approximately low priced cars.

##### Function to plot box plot and print the number of observation on box plots

In [ ]:
def numberonboxplot(df,col,ax):
    tops = df.groupby([col])['price'].median().sort_values(ascending=True)
    nobs = pd.DataFrame(pd.concat([df.groupby([col])['price'].median().sort_values(ascending=True),
                 df[col].value_counts()],axis=1))
    nobs.sort_values(by='price',ascending=True,inplace=True)
    nobs= nobs[col].values
    nobs = [str(x) for x in nobs.tolist()]
    nobs = ["n: " + i for i in nobs]
    # Add it to the plot
    #print(nobs)
    pos = range(len(nobs))
    
    for tick,label in zip(pos,ax.get_xticklabels()):
        ax.text(pos[tick], tops[tick] + 0.03, nobs[tick],
                horizontalalignment='center', size='medium', color='black', weight='semibold')
        
def plotBoxPlot(col):   
    # Plot a Horizontal Line at mean price of complete dataset
    plt.axhline(y=car['price'].mean(), color='r')

    # Order the box plots in acending order of median value
    dfm = pd.DataFrame(car[[col,'price']].groupby([col]).median().sort_values(by='price',ascending=True))
    l =list(dfm.index.get_values())
    ax = sns.boxplot(x=col, y='price', data=car, palette="Set1",order=l)
    # Add jitter with the swarmplot function.
    sns.swarmplot(x=col , y='price', data=car, color="blue",order=l)

    if col!='symboling':
        numberonboxplot(car,col,ax)

In [ ]:
cat_feature = ['fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','symboling',
               'enginetype','cylindernumber','fuelsystem']
plt.figure(figsize=(20,25))
for i, feature in enumerate(cat_feature):
    if(len(cat_feature)%2 ==0):
        rows = int(len(cat_feature)/2)
    else:
        rows = int(len(cat_feature)/2) +1
    
    plt.subplot(rows, 2, i+1)
    # Draw a horizontal Line - mean amount of price
    plt.axhline(y=car['price'].mean(), color='r')
    
    # Order the box plots in acending order of median value
    dfm = pd.DataFrame(car[[feature,'price']].groupby([feature]).median().sort_values(by='price',ascending=True))
    l =list(dfm.index.get_values())
    
    ax = sns.boxplot(x=feature, y='price', data=car, palette="Set1", order=l)
    
    # Add jitter with the swarmplot function.
    sns.swarmplot(x=feature, y='price', data=car, color="blue", order=l)
    
    # Show Number of Observation On the Box plot
    if feature!='symboling':
        numberonboxplot(car,feature,ax)
    
plt.tight_layout()
plt.show()


#### Box plot based on car company and price distribution

In [ ]:
print(car.price.describe())
plt.figure(figsize=(25,15))
plt.axhline(y=car['price'].mean(), color='r')

#Order the box plots in acending order of median value
dfm = pd.DataFrame(car[['companyName','price']].groupby(['companyName']).median().sort_values(by='price',ascending=True))
l =list(dfm.index.get_values())
ax = sns.boxplot(x='companyName', y='price', data=car, palette="Set1",order=l)

# Add jitter with the swarmplot function.
sns.swarmplot(x='companyName' , y='price', data=car, color="blue",order=l)

numberonboxplot(car,'companyName',ax)

Findings:

    - Most Of the features are having overlapping information. 
    - None of the features alone is explaining the cause of pricing.
    - For car companies such as mercury, audi, bmw, jaguar, buick, porsche,volvo - the car with minimum amount is higher than the population mean amount.
    - We can use LableEncoding with ascending mean amount for model building.
    - Based on the observations, we can't use all the categories for analysis as it may result in wrong conclusion due to high class imbalance
    - We need to group few categories and find some derived features to check the variance

#### By Grouping drivewheel, carbody and getting the mean price, it can be clearly visualized that rear wheel drive is on an average having higher value for all car body type.

In [ ]:
# grouping results
car_gp = car[['drivewheel','carbody','price']]
car_gp1 = car_gp.groupby(['drivewheel','carbody'],as_index=False).mean()
pivot_by_wheeltype = car_gp1.pivot(index='drivewheel',columns='carbody')
print(pivot_by_wheeltype.fillna(0))

sns.heatmap(pivot_by_wheeltype, annot=True, fmt='.2f',cmap='coolwarm')

Finding :
    - rwd - Rear wheel drive car on an average higher price than 4 wheel drive and forward wheel drive cars. 

In [ ]:
car_gp2 = car[['drivewheel','companyName','price']]
car_gp2 = car_gp2.groupby(['drivewheel','companyName'],as_index=False).min()
car_gp2 = car_gp2.sort_values(by='price', ascending= True)


print('Average car price of the whole data set:', car['price'].mean())
print('\n')
print('Car Companies who manufactures cars with price much higher than average car price: ')
car_gp2[car_gp2['price'] >= car['price'].mean()]

### B - Understanding the numerical variables
- Correlation Analysis of Numerical Columns to identify highly correlated features
- Pair plots to check the relationship of Price against other numerical columns
- Price vs Numerical Features Pair plot with regression line
- Ditribution and box Plot of Car Price
- Box Plots or Caterical Fetaures against Car Price
- Binning of Car Price/Horsepower based on price Range as Low, Mid and High
- Distribution plot based on Car Price Range against other numerical Features

In [ ]:
'''
Get the numerical columns only
'''
cat_car = car.select_dtypes(include=['object'])
num_cols = set(car.columns).difference(set(cat_car.columns))
num_cols = list(num_cols)

#Check The Min, Max, Average of numerical features
car[num_cols].describe().transpose()

#### i) Correlation Analysis Using Heatmap
- citympg and highwaympg are highly correlated > .97 and both are having a -ve(almost equal) correlation with Price, Horsepower and engine size. 
- Carlength is having a high +ve correlation with Carwidth(.84), Carwheel base(.87)
- Price is having +v correlation with curbweight, enginesize, horsepower, carwidth, carlength, wheelbase, boreratio
- Highly Correlated Independent Features are 
    - 1) citympg and highwaympg
    - 2) curbweight, Carwidth, wheelbase and carlength
    - 3) Enginesize and horsepower
    - 4) Enginesize, curbweight, wheelbase and carlength


We can eleminate few of the features after performing the P-value test, RFE and VIF


In [ ]:
plt.figure(figsize=(25,20))
sns.set(font_scale=2)
sns.heatmap(car[num_cols].corr(),annot=True,cmap='coolwarm')

#### ii) Analysis of Numerical features using Pair Plot
- It is obvious that Price is having a linear relation ship with folliwng features:
    - curbweight 
    - enginesize
    - carlength
    - highwaympg
    - horsepower
    - wheelbase
    - carwidth
    - citymapg
    - bore ratio
    
- It seems there are two categories based on car compressionratio distribution but Price is not having linear correlation with compression ratio.

#### Numerical features such as peakrpm, stroke, carheight and compressionratio don't seem like having any linear realtionship with Price

In [ ]:
plt.rcParams['figure.figsize'] = (20,20)
#plt.figure(figsize=(24,16))
num_cols.remove('price')
sns.set(font_scale=1)
sns.pairplot(data=car, x_vars=num_cols[0:4], y_vars='price',kind="reg")
sns.pairplot(data=car, x_vars=num_cols[4:8], y_vars='price',kind="reg")
sns.pairplot(data=car, x_vars=num_cols[8:12], y_vars='price',kind="reg")
sns.pairplot(data=car, x_vars=num_cols[12:16], y_vars='price',kind="reg")
#sns.pairplot(data=car, x_vars=num_cols[12:16], y_vars='price',kind="reg")

In [ ]:
### Scatter plot of price and engine size
sns.lmplot(y='price',x="enginesize", data=car);
plt.title('Engine Size vs Price')


In [ ]:
sns.lmplot(y='price',x="curbweight", data=car)
plt.title('Curb weight vs Price')

Findings:
    - Price is having a linear relationship with Curbweight and EngineSize

In [ ]:
sns.lmplot(y='citympg',x='curbweight', data=car, fit_reg=True); #hue="companyName",
sns.lmplot(y='citympg',x='enginesize', data=car, fit_reg=True);

Findings :
    - Citympg decreases with increase in enginesize and curbweight

In [ ]:
sns.lmplot(x= 'enginesize' , y='price', hue = 'doornumber', data=car)

In [ ]:
sns.lmplot(x= 'enginesize' , y='price', hue = 'fuelsystem', data=car)

sns.lmplot(x= 'horsepower' , y='price', hue = 'fuelsystem', data=car)

sns.lmplot(x= 'enginesize' , y='price', hue = 'symboling', data=car)

sns.lmplot(x= 'horsepower' , y='price', hue = 'symboling', data=car)

sns.lmplot(x= 'enginesize' , y='price', hue = 'cylindernumber', data=car)

Findings:
 - Door Number is not contributing significatnt variation to the price
 - Fuel system and horsepower or enginetype expains the price variation pretty well.
 - Horse Power and symboling(riskfactor) is explaining some of the price variation but not that clear.

#### iii) Distribution and box plot of amount

In [ ]:
plt.rcParams['figure.figsize'] = (10,4)

fig, ax =plt.subplots(1,2, figsize=(10,4),squeeze=False)
s1 = sns.distplot(car.price,bins = 10, ax=ax[0][0])
#plt.show()

s2= sns.boxplot(y=car.price, ax=ax[0][1])
plt.show()

#### iv) Binning of Car Dataset based on Price and Horsepower and plot bar plots:

##### New metrics are derived if applicable and are used for analysis and modelling.

- Based on the car price, we can segregate cars into three categorical segments:
    - Low
    - Mid
    - High
- Based on horse power, the dataset can be segregated into three bins:
    - Low
    - Mid
    - High

In [ ]:
bins = np.linspace(min(car["horsepower"]), max(car["horsepower"]), 4)
labels = ['Low','Mid','High']
car['horsepower_range'] = pd.cut(car.horsepower,bins=bins,labels=labels,include_lowest=True )

bins = np.linspace(min(car["price"]), max(car["price"]), 4)
labels = ['Low','Mid','High']
car['price_range'] = pd.cut(car.price,bins=bins,labels=labels,include_lowest=True)
'''
Print the count of cars in each category
'''
# There is not enough data in each category.
print(car["price_range"].value_counts())
print(car["horsepower_range"].value_counts())

- curbweight is correlated with carheight and carwidth so, we can derive new matrices by applying some generic formulas such deriving volume and desity
- horsepower and enginesize are correalted hence we can create a new feature as powerper volume size of the engine


In [ ]:
# Generate new features
car['volume'] = car['carheight'] * car['carwidth'] * car['carlength']
car['density'] = car['curbweight'] / car['volume']
car['powerpervolume'] = car['horsepower'] / car['enginesize']

# Repopulate the Num_Cols List
#num_cols = set(car.columns).difference(set(cat_car.columns))
#num_cols = list(num_cols)

num_cols.append('volume')
num_cols.append('powerpervolume')
num_cols.append('density')

In [ ]:
num_cols

##### Plot bar plots to check the counts based on price and horsepower range

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1, 2,1)
plt.bar(labels,car["price_range"].value_counts())
plt.xlabel("Price")
plt.ylabel("count")
plt.title("Price Range")

plt.subplot(1, 2, 2)
plt.bar(labels,car["horsepower_range"].value_counts())
plt.xlabel("Price")
plt.ylabel("count")
plt.title("Horsepower Range")

#### v) Distribution Plot Price_range vs Other Numerical Features
- The distribution is overlapping hence not that helpful in interpreting them.

In [ ]:
bins = 5
plt.figure(figsize=(18,18))
for i, feature in enumerate(num_cols):
    if len(num_cols) %2 == 0:
        rows = int(len(num_cols)/2)
    else:
        rows = int(len(num_cols)/2) +1

    plt.subplot(rows, 2, i+1)
    
    sns.distplot(car[car.price_range=='Low'][feature], bins=bins, color='red', label='L');
    sns.distplot(car[car.price_range=='Mid'][feature], bins=bins, color='blue', label='M');
    sns.distplot(car[car.price_range=='High'][feature], bins=bins, color='green', label='H');
    
    plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


#### vi) Distribution Plot Horsepower_range vs Other Numerical Features
- The distribution is overlapping hence not that helpful in interpreting them.

In [ ]:
bins = 5
plt.figure(figsize=(15,15))
for i, feature in enumerate(num_cols):
    if len(num_cols) %2 == 0:
        rows = int(len(num_cols)/2)
    else:
        rows = int(len(num_cols)/2) +1
    
    plt.subplot(rows, 2, i+1)
    
    sns.distplot(car[car.horsepower_range=='Low'][feature], bins=bins, color='red', label='L');
    sns.distplot(car[car.horsepower_range=='Mid'][feature], bins=bins, color='blue', label='M');
    sns.distplot(car[car.horsepower_range=='High'][feature], bins=bins, color='green', label='H');
    
    plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

### P-Value Analysis - Pearsonr  Correlation Analysis
- Approach :
 - If p-value is  <  0.001: then strong evidence that the correlation is significant.
 - p-value is  <  0.05: then moderate evidence that the correlation is significant.
 - p-value is  <  0.1: then weak evidence that the correlation is significant.
 - p-value is  >  0.1: then no evidence that the correlation is significant.

In [ ]:
from scipy import stats

for col in num_cols: 
    pearson_coef, p_value = stats.pearsonr(car[col], car['price'])
    
    print(col,':')
    print("The PearsonR between feature and price is {} with a P-value = {}".format(pearson_coef, round(p_value,5)))
    
    if p_value < 0.001:
        print('Correlation between feature and price is statistically significant. P-value less than .001.')
    elif p_value < 0.05:
        print('Correlation between feature and price is statistically significant.')
    elif p_value < 0.1:
        print('Correlation between feature and price is statistically weak.')
    else:
        print('Correlation between feature and price is statistically not significant.')
    print('\n')


## Dummy variables creation wherever applicable

##### 1 - Dealing with categorical columns having two values : doornumber, aspiration and fueltype. 
    - Encoding with 0 and 1.
    - doornumber is can be converted to numeric value four to 4 and two to 2.

In [ ]:
print(pd.concat([car['doornumber'].value_counts(),car['doornumber'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

print(pd.concat([car['aspiration'].value_counts(),car['aspiration'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

print(pd.concat([car['fueltype'].value_counts(),car['fueltype'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

bin_cat = {"doornumber":{"four": 4, "two": 2},
           "aspiration":{"std":1,"turbo":0},
           "fueltype":{"gas":1,"diesel":0}}

car.replace(bin_cat, inplace=True)

##### 2 - Dealing with cylindernumber categorical column

In [ ]:
print(pd.concat([car['cylindernumber'].value_counts(),car['cylindernumber'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

#Create Dummies for Four, Six, Five  CylinderNumber and Remove other columns
car= pd.get_dummies(car,columns=['cylindernumber'])
car.drop(['cylindernumber_eight','cylindernumber_two','cylindernumber_twelve','cylindernumber_three'],axis=1,inplace=True)

- As you can see there are less than 5 % of data for few categories, we can't use them for modelling what we will do, we first apply dummy to create individual columns for each cylinder type then drop the columns correspoding to cylinders :eight (2.54 %),two (1.52%),twelve 0.51% and three  0.51%.

##### 3 - Dealing with carbody categorical column

- As you can see there are less than 5 % of data for few categories, we can't use them for modelling. What we will do, we first apply dummy to create individual columns for each carbody type then drop the columns correspoding to carbody :hardtop(3.55%) and convertible(2.54%)

In [ ]:
print(pd.concat([car['carbody'].value_counts(),car['carbody'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

# Create Dummies and drop hardtop and convertible
car= pd.get_dummies(car,columns=['carbody'])
car.drop(['carbody_hardtop','carbody_convertible'],axis=1,inplace=True)

##### 4 - Dealing with drivewheel categorical column
- Remove 4wd category as having only 4.57% of the data after performing one hot encoder

In [ ]:
print(pd.concat([car['drivewheel'].value_counts(),cat_car['drivewheel'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

# Create Dummies and Drop 4wd
car= pd.get_dummies(car,columns=['drivewheel'])
car.drop(['drivewheel_4wd'],axis=1,inplace=True)

##### 5 - Dealing with enginelocation categorical column
- Remove rear enginelocation category as having only 1.02% of the data by performing one hot encoder i.e front =1, rear =0

In [ ]:
print(pd.concat([car['enginelocation'].value_counts(),car['enginelocation'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

# Update EngineLocation
bin_cat = {"enginelocation":{"front": 1, "rear": 0}}
car.replace(bin_cat, inplace=True)

##### 6 - Dealing with enginetype categorical column

- The engine type of a vehicle states how the engine is configured or its operations in term of design of valves, cam shaft and cylinders. Here there are seven engine types: 
    - ohc (OverHead Cam) - 73.60%
    - dohc (Dual OverHead Cam) - 5.08%
    - dohcv (Dual OverHead Cam and Valve) - 0.51%
    - l (L engine), ohc (OverHead Cam) - 5.58%
    - ohcf (OverHead Cam and Valve F engine) - 7.11%
    - ohcv (OverHead Cam and Valve) - 6.6%
    - rotor (Rotary engine) - 1.52%
    
- Here 'dohcv' is having one records and its withing the range of 'dohc' hence update the category type to 'dohc'
- After applying Onehotencoding remove the column correspoding to 'rotor' enginetype 

In [ ]:
print(pd.concat([car['enginetype'].value_counts(),car['enginetype'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))
plt.figure(figsize=(10,5))
# Box Plot See the dictribution for each Engine Type
plotBoxPlot('enginetype')

# Update dohcv to dohc
eng_upd = {"enginetype":{"dohcv": "dohc"}}
car.replace(eng_upd, inplace=True)

# Create Dummies
car= pd.get_dummies(car,columns=['enginetype'])

# Drop Rotor
car.drop(['enginetype_rotor'],axis=1,inplace=True)

##### 7 - Dealing with fuelsystem categorical column
- Fuel System 1bbl and 2bbl the dictibution is almost same - Lets update them as 2bbl
- Fuel system spfi having only 1 data and price is just higher than the mean of spdi. Hence update it to spdi. 
- Fuel system 4bbl and mfi having only 2 and 1 data respectively and have price just less than or equal to mean of idi. Hence update them to idi
- Apply OnehotEncoding and then drop the column correspoding to spdi(as having least no of values)

In [ ]:
print(pd.concat([car['fuelsystem'].value_counts(),car['fuelsystem'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

# Plot box plot for fuelsystem type distribution
plt.figure(figsize=(10,5))
plotBoxPlot('fuelsystem')

# Update dohcv to dohc
fuel_upd = {"fuelsystem":{"1bbl": "2bbl","spfi":"spdi","4bbl":"idi","mfi":"idi"}}
car.replace(fuel_upd, inplace=True)

#Check the dictribution after update
plt.figure(figsize=(10,5))
plotBoxPlot('fuelsystem')

# Create Dummies
car= pd.get_dummies(car,columns=['fuelsystem'])

# Drop Rotor
car.drop(['fuelsystem_spdi'],axis=1,inplace=True)

##### 8 - Split Engine Size to Low Mid High

In [ ]:
plt.figure(figsize=(20,15))
# Plot a Horizontal Line at mean price of complete dataset
plt.axhline(y=car['enginesize'].mean(), color='r')

# Order the box plots in acending order of median value
dfm = pd.DataFrame(car[['companyName','enginesize']].groupby(['companyName']).median().sort_values(by='enginesize',ascending=True))
l =list(dfm.index.get_values())
ax = sns.boxplot(x='companyName', y='enginesize', data=car, palette="Set1",order=l)
# Add jitter with the swarmplot function.
sns.swarmplot(x='companyName' , y='enginesize', data=car, color="blue",order=l)

#if col!='symboling':
    #numberonboxplot(car,'companyName',ax)

In [ ]:
# Create Bins as Engine Size
bins = [0,100,150,500]
labels = [0,1,2]
car['engine_range'] = pd.cut(car.enginesize,bins=bins,labels=labels,include_lowest=True )
plt.figure(figsize=(10,6))
plotBoxPlot('engine_range')

##### 9 - Split Horse Power to High and Low

In [ ]:
plt.figure(figsize=(20,15))
# Plot a Horizontal Line at mean price of complete dataset
plt.axhline(y=car['horsepower'].mean(), color='r')

# Order the box plots in acending order of median value
dfm = pd.DataFrame(car[['companyName','horsepower']].groupby(['companyName']).median().sort_values(by='horsepower',ascending=True))
l =list(dfm.index.get_values())
ax = sns.boxplot(x='companyName', y='horsepower', data=car, palette="Set1",order=l)
# Add jitter with the swarmplot function.
sns.swarmplot(x='companyName' , y='horsepower', data=car, color="blue",order=l)

#if col!='symboling':
    #numberonboxplot(car,'companyName',ax)

In [ ]:
# Create Bins as per hosrse power and car type distribution
bins = [0,150,300]
labels = [0,1]
car['horsepower_range'] = pd.cut(car.horsepower,bins=bins,labels=labels,include_lowest=True )

plt.figure(figsize=(10,6))
plotBoxPlot('horsepower_range')

##### 10 - Dealing with companyName categorical column
- Apply Dummy to do one hot encoding on company name

In [ ]:
print(pd.concat([car['companyName'].value_counts(),car['companyName'].
                 value_counts(normalize=True).apply(lambda x: round(x*100,2))],axis=1))

# Plot box plot for fuelsystem type distribution
plt.figure(figsize=(20,15))
plotBoxPlot('companyName')


# Keep a Back Up Copy for further Model Testing as we may need to encode based on car company name
carbackup = car

# Create Dummies
car= pd.get_dummies(car,columns=['companyName'],drop_first=True)



In [ ]:
# Drop the columns price_range
car.drop(['price_range'],axis=1,inplace=True)

## Model Building
### Approach:
    - Train test split
    - Build different models with complete data to understand the p-value, VIF
    - Perform RFE to further eliminate features
    - Identify the features for final model building
    - Train the model with Train Set
    - Check the R2 and Adjusted R2 on both train and test set
    - Perform Residual Analysis

### Function for train test split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
def prepareTrainTestSets(car,randomness=42):
    '''
    Step 1: Train Test Split
    '''    
    np.random.seed(0)
    df_train, df_test = train_test_split(car, train_size = 0.8, test_size = 0.2, random_state = randomness)

    '''
    Step 2: Scale the training data
    '''
    scaler = MinMaxScaler()
    all_cols = list(car.columns)
    df_train[all_cols] = scaler.fit_transform(df_train[all_cols])

    '''
    Step 3: Create the X_train and y_train
    '''
    y_train = df_train.pop('price')
    X_train = df_train
        
    '''
    Setp 4: Scale Test Set and create the X_test and y_test
    '''
    # Scale Test Set
    df_test[all_cols] = scaler.transform(df_test[all_cols])
    y_test = df_test.pop('price')
    X_test = df_test
    
    return y_train,X_train,y_test,X_test

In [ ]:
# Get the Train and Test Dataframe
y_train,X_train,y_test,X_test = prepareTrainTestSets(car)

# Check the shape
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

## Model Building and Test Evaluations 

### Model 1: Using Sklearn LinearRegression

In [ ]:
'''
Step 1: Train the Model the model using Training Set
'''
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
#num_cols.remove('price')

# fit the model to the training data
lr.fit(X_train,y_train)

######  Evaluation Using R-Square Value

In [ ]:
'''
Step 2: Predict and Evaluate the training MSE and R-Square
'''
from sklearn.metrics import mean_squared_error, r2_score

#Make Prediction Using the test set
y_hat = lr.predict(X_train)
mse = mean_squared_error(y_train, y_hat)
r_squared = r2_score(y_train, y_hat)

print('Model 1 Evaluation:')

print('\tTrain Mean_Squared_Error :' ,mse)
print('\tTrain R_square_value :',r_squared)

'''
Step 3: Predict and Evaluate the training MSE and R-Square
'''
y_hat_test = lr.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
r_squared = r2_score(y_test, y_hat_test) 

print('\tTest Mean_Squared_Error :' ,round(mse,3))
print('\tTest R_square_value :',round(r_squared,3))


#### Finding: 
    - When all the features are used, the model is slightly overfitting i.e. it is performing well on train data but not so good in test data.
    - So let's build another model with all the features but using Statsmodel OLS library to get the P-Values for each feature. Before that, let's identify 15 significant features using RFE.

#### RFE - Recursive feature Elemination

**Function** To Select first the 15 features which seems significant.

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def getColForAnalysisUsingREF(X_train, y_train, featureCount,colstodrop):
    # Running RFE with the output number of the variable equal to 20
    lm = LinearRegression()
    X_train_n = X_train.drop(colstodrop,axis=1)
    lm.fit(X_train_n, y_train)

    rfe = RFE(lm, featureCount)             # running RFE
    rfe = rfe.fit(X_train_n, y_train)

    # Print the Output of RFE
    print('RFE Output:\n' , list(zip(X_train_n.columns,rfe.support_,rfe.ranking_)))

    # Create the columns to include the significant features identified by RFE
    colsForAnalysis = []
    for i,j,k in list(zip(X_train_n.columns,rfe.support_,rfe.ranking_)):
        if j:
            colsForAnalysis.append(i)
    return colsForAnalysis

Findings: 
- The test R-Square 0.87 is much lower than the Train R-Square 0.97. This is a case of slight overfitting i.e. the model is fitting the training data but unable to predict on test data. Hence we need to Check p-value and vif for selecting the features.
- Let's tune the model.


## Function for Model Building using statsmodel OLS and evaluation

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from tabulate import tabulate


from sklearn.metrics import mean_squared_error, r2_score

def buildModel(colstodrop,y_train,X_train,y_test,X_test):
    
    #y_train,X_train,y_test,X_test = prepareTrainTestSets(car)
    
    '''
    Step 1: Train the model
    '''
    if not colstodrop:
        X_m = X_train
    else:
        X_m = X_train.drop(colstodrop,axis=1)
    X_m = sm.add_constant(X_m)
    # print('shape X_m after constant:',X_m.shape)
    lm = sm.OLS(y_train,X_m).fit()
    #print(lm.summary(),'\n') 
    y_hat_train = lm.predict(X_m)

    '''
    Step 2: Get the VIF
    '''
    # Create a dataframe that will contain the names of all the feature variables and their respective VIFs
    vif = pd.DataFrame()
    if not colstodrop:
        X_m = X_train
    else:
        X_m = X_train.drop(colstodrop,axis=1)
    if len(X_m.columns) > 1:
        vif['Features'] = X_m.columns
        vif['VIF'] = [variance_inflation_factor(X_m.values, i) for i in range(X_m.shape[1])]
        vif['VIF'] = round(vif['VIF'], 2)
        vif = vif.sort_values(by = "VIF", ascending = False)
        #print(vif)
    
    '''
    Step 3: Predict On Test Set
    '''
    #Predict on test data and Evaluate
    # print('shape X_test before constant:',X_test.shape)
    if not colstodrop:
        X_test_m = X_test
    else:
        X_test_m = X_test.drop(colstodrop,axis=1)
    # print('shape X_test_m before constant:',X_test_m.shape)
    X_test_m = sm.add_constant(X_test_m, has_constant='add')
    # print('shape X_test_m after constant:',X_test_m.shape)
    y_hat = lm.predict(X_test_m)
    mse = mean_squared_error(y_test, y_hat)
    r_squared = r2_score(y_test, y_hat) 
    
    '''
    Step 4: Print the Results
    '''
    print('Training Results :')
    print('F-statistics :', lm.fvalue)
    print('R-Squared :', lm.rsquared)
    print('Adj R-Squared :', lm.rsquared_adj)
    print('\n')
    print('Testing Results :')
    print('Test Mean_Squared_Error :' ,mse)
    print('Test R_square_value :',r_squared)
    print('\n')
    
    '''
    Step 5: Create the View for P-value, VIF and Coefficients
    '''
    
    #Create a dataframe to hold the p-values 
    d = lm.pvalues.apply(lambda x:round(x,3)).to_dict()
    p_val = pd.DataFrame()
    p_val['Features'] = list(d.keys())
    p_val['p-value'] = list(d.values())
    
    ##Create a dataframe to hold the p-values and VIF by mergeing the dataframes
    # For Const the VIF will be zero. Or in case of single variable, VIF won''t be calculated.
    if vif.empty ==False:
        p_val = p_val.merge(vif, on ='Features', how='left')
        p_val.fillna(0,inplace=True)
        p_val.sort_values(by='p-value',ascending=False,inplace=True)
    
    print('P-value and VIF values for the features:')
    
    #Create and dataframe for Co-effs
    c = lm.params.apply(lambda x:round(x,3)).to_dict()
    coef = pd.DataFrame()
    coef['Features'] = list(c.keys())
    coef['Coeffs'] = list(c.values())
    pvif = p_val.merge(coef, on ='Features', how='left')
    pvif.fillna(0,inplace=True)
    
    print(tabulate(pvif, headers='keys', tablefmt='psql'))
    
    return lm, pvif, y_hat_train, y_hat 

### Model 2 : Using StatsModel Oridnary Least Square to check p-values for all the features
- Customised output for readability and better
- P-value, VIF and Co-effs on a single dataframe
- Use tabulate libaracy to display the dataframe in  much clearer way

In [ ]:
# Perform Multilinear Regression using All Columns without RFE
y_train,X_train,y_test,X_test = prepareTrainTestSets(car)

colstodrop = []
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Drop Highly Correlated COlumns
# From Earlier Heat map we identified that following features are highly correlated
colstodrop = ['volume','carlength','citympg','curbweight']

# Identify columns using RFE
colsForAnalysis = getColForAnalysisUsingREF(X_train, y_train,20,colstodrop)
print('\n')
print('Significant columns after performing RFE :\n' , colsForAnalysis)

# Find the columns to drop excluding the columns identified as significant by RFE 
colstodrop = list(set(X_train.columns).difference(set(colsForAnalysis)))

colstodrop.append('volume')
colstodrop.append('carlength')
colstodrop.append('citympg')
colstodrop.append('curbweight')

Manual Tuning:
    - The model is not performing well n the test set. This is clear case of overfitting.
    - Drop the columns having High P-value and High VIF
    - Then High P and Low VIF remove these first
    - Then Low P and High VIF remove these after the ones above 
    - check P-values and Correlation coefficient to select the features

### Model 3 :Including the features identified by REF and perform Interations

##### Including the features identified in RFE only

In [ ]:
# create dataframe to hold the rediuals
train_res = pd.DataFrame()
test_res = pd.DataFrame()

# Perform the Train and Test
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m0'] = y_train - y_hat_train
test_res['m0'] = y_test - y_hat_test

#### Findings: Seems bit overfitted on test data

### Check the Correlation between numerical features:

Remember: Correlation is not Causation. So, need to exclude the features considering P-Value and Vif along with correlation.
 - Due to high correlation We have already removed these features before performing RFE 
     - volume
     - carlength
     - citympg
     - curbweight

##### Excluding the other features which are not identified

In [ ]:
plt.figure(figsize=(18,15))
sns.set(font_scale=1)
sns.heatmap(X_train.drop(colstodrop,axis=1).corr(),annot=True)

#### Conclusion From Heatmap:
From the HeatMap, we can conclude that:
        - companyName_subaru highly correlated with enginetype_ohcf

##### First Remove the Highly Correlated Fetaures

In [ ]:
# First Remove the Highly Correlated Fetaures
colstodrop.append('companyName_subaru')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m1'] = y_train - y_hat_train
test_res['m1'] = y_test - y_hat_test


##### Rebuild the Model by excluding the features having high p-value and vif
- Remove the features:
    - having high P-values p > .05 (Priority 1)
    - having high p-value and high vif(priority 2)
    - low p-value and high VIF(priority 3)

Repreat the iteration untill we have p-value less than 5% and VIF less than 5 for the selected features.


##### Interation 1: Drop powerpervolume high p-value

In [ ]:
colstodrop.append('powerpervolume')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m2'] = y_train - y_hat_train
test_res['m2'] = y_test - y_hat_test

##### Interation 2: Drop companyName_renault high P

In [ ]:
colstodrop.append('companyName_renault')

lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)
# Populate the Residuals
train_res['m3'] = y_train - y_hat_train
test_res['m3'] = y_test - y_hat_test

##### Interation 3: Drop enginetype_ohcf as very high P-value

In [ ]:
colstodrop.append('enginetype_ohcf')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)
# Populate the Residuals
train_res['m4'] = y_train - y_hat_train
test_res['m4'] = y_test - y_hat_test

##### Interation 4: Drop enginelocation high VIF

In [ ]:
colstodrop.append('enginelocation')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m5'] = y_train - y_hat_train
test_res['m5'] = y_test - y_hat_test

##### Interation 5: Drop cylindernumber_six p_value

In [ ]:
colstodrop.append('cylindernumber_six')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m6'] = y_train - y_hat_train
test_res['m6'] = y_test - y_hat_test

##### Interation 6: Drop wheelbase high VIF

In [ ]:
colstodrop.append('wheelbase')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m7'] = y_train - y_hat_train
test_res['m7'] = y_test - y_hat_test

In [ ]:
# By Dropping wheelbase performance decreases hnece keep it

In [ ]:
colstodrop.remove('wheelbase')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m7'] = y_train - y_hat_train
test_res['m7'] = y_test - y_hat_test

##### Interation 7: Drop carwidth high VIF

In [ ]:
colstodrop.append('carwidth')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m8'] = y_train - y_hat_train
test_res['m8'] = y_test - y_hat_test

##### Interation 8: Drop density high VIF

In [ ]:
colstodrop.append('density')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m9'] = y_train - y_hat_train
test_res['m9'] = y_test - y_hat_test

##### Interation 9:Drop companyName_peugeot

In [ ]:
colstodrop.append('companyName_peugeot')
#colstodrop.remove('fueltype')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m10'] = y_train - y_hat_train
test_res['m10'] = y_test - y_hat_test

##### Interation 10: Include companyName_honda high P

In [ ]:
colstodrop.append('companyName_honda')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m11'] = y_train - y_hat_train
test_res['m11'] = y_test - y_hat_test

##### Interation 11: Exclude boreratio high P

In [ ]:
colstodrop.append('boreratio')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m12'] = y_train - y_hat_train
test_res['m12'] = y_test - y_hat_test

##### Interation 12: Include enginesize from exclusion

In [ ]:
colstodrop.remove('enginesize')
lm,pvif,y_hat_train,y_hat_test = buildModel(colstodrop,y_train,X_train,y_test,X_test)

# Populate the Residuals
train_res['m13'] = y_train - y_hat_train
test_res['m13'] = y_test - y_hat_test

# Final Model Summary Stats

In [ ]:
print(lm.summary())

### Residual Analysis of the train data

In [ ]:
cols = train_res.columns
plt.figure(figsize=(25,30))
for i, col in enumerate(cols):
    
    if len(cols) %2 == 0:
        rows = int(len(num_cols)/2)
    else:
        rows = int(len(num_cols)/2) +1
    
    plt.subplot(rows, 2, i+1)    
    ax = sns.distplot(train_res[col],bins = 20)
    ax.set(xlabel='Train Errors in model:' + col)

    plt.tight_layout()
plt.show()

Finding: 
- Error is uniformly distributed

## Model Evaluation

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_train,(y_train - y_hat_train))
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,(y_test - y_hat_test))
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)

There is no such pattern identified from the scatter plt. So we can infer that the data is linear and there is no Heteroskedasticity issue from the model

# Conclusion:
From the HeatMap, we can conclude that:
        - Wheel base , Car Length , Car Width are highly correlated.
        - Highway mpg and city mpg are highly correlated.
        - Compression ratio and fuel type are correlated
        - Engine size, Curb weight and horse power are also correlated


From the Scatter/Pair plots, features which have stronger relationship with price are:
    - 1. Curb-Weight
    - 2. Engine-Size
    - 3. Horsepower
    - 4. Mpg(City / Highway mpg)
    - 5. Lenght/ Width
    - 6. Engine Type
    - 7. Fuel System
    - 8. Number Of Cylinders - 4 or 6

#### Identification of factors for varying car price and help Chinese company to enter US market:
- Enginesize, curb-weight, horsepower - These parameters have positive relationship with car price.
- Mileage per gallon: As the MPG increases the car price decreases. It means, MPG has a -ve relationship with Horsepower, Enginesize, and Curb weight. It mean, more powerful is the engine, it will consume more Diesel/Gas. Hence the mileage decreases with increase in engine size.
- Fuel Type Diesel Cars are manufactured more in comparision with Gas cars.
- Std aspirations type are more in the market when compared against the Turbo aspiration type.
- Forward Wheel Drive(Fwd) cars are manuafactured more over the 4 Wheel Drive and Rear Wheel Drive Cars. Fwd cars are almost are comparatively lower priced than Rwd cars.
- MPFI Fuel System car is manuafactured more and It is bit costlier than other fule systems. 2bbl fuel system is the 2nd highest manuafactured fuel system type and it is reasonably priced. 
- Engineype OHC is manufactured more when compared against other car engine types.


### Final Model Report Explaining the reason for pricing are:
For these the P-value and VIF is less tha 5%
- companyName_plymouth
- companyName_dodge
- companyName_mitsubishi
- companyName_toyota
- wheelbase
- horsepower
- companyName_bmw
- companyName_buick
- companyName_porsche

##### Training Results :
    - F-statistics : 140.05938256007505
    - R-Squared : 0.8955620583731947
    - Adj R-Squared : 0.8891678986817576

- Testing Results :
    - Test Mean_Squared_Error : 0.014222211103648302
    - Test R_square_value : 0.8761129569542494

**Further We Can Include EngineSize and CurbWeight To Explain More Variation explaining the pricing detail**.

- After including EngineSize:

- Training Results :
    - F-statistics : 137.07329010374917
    - R-Squared : 0.9037404674876298
    - Adj R-Squared : 0.897147348822399

- Testing Results :
    - Test Mean_Squared_Error : 0.011584866326137502
    - Test R_square_value : 0.8990863781471152